In [122]:
%run dados.ipynb

In [123]:
spark = (SparkSession.builder
        .master("local")
        .appName("projetosoulcode")
        .config('spark.jars','gcs-connector-hadoop3-latest.jar' )
        .config('spark.hadoop.fs.gs.impl','com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem')
        .config('spark.hadoop.fs.gs.auth.service.account.enable', 'true')
        .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile",'soulcodeproject-44ce61622773.json')
        .config('spark.hadoop.fs.gs.project.id','soulcodeproject')
        .getOrCreate())
spark.conf.set("spark.sql.caseSensitive", "true")

In [274]:
schema = (
    StructType([
        StructField("ID",IntegerType(), False),
        StructField("AnoNascimento", IntegerType(), False),
        StructField("GrauInstrução", StringType(), True),
        StructField("EstadoCivil", StringType(), True),
        StructField("Renda", DoubleType(), True),
        StructField("NumCrianca", IntegerType(), True),
        StructField("NumAdolescente", IntegerType(), True),
        StructField("DataConta", StringType(), True),
        StructField("UltimaCompra", IntegerType(), True),
        StructField("QtdVinho", IntegerType(), True),
        StructField("QtdFrutas", IntegerType(), True),
        StructField("QtdCarne", IntegerType(), True),
        StructField("QtdPeixe", IntegerType(), True),
        StructField("QtdDoces", IntegerType(), True),
        StructField("QtdOuro", IntegerType(), True),
        StructField("QtdComprasDesconto", IntegerType(), True),
        StructField("QtdComprasWeb", IntegerType(), True),
        StructField("QtdCompraCatalogo", IntegerType(), True),
        StructField("QtdCompraLoja", IntegerType(), True),
        StructField("NumVisitasSite", IntegerType(), True),
        StructField("AceitouCam3", IntegerType(), True),
        StructField("AceitouCam4", IntegerType(), True),
        StructField("AceitouCam5", IntegerType(), True),
        StructField("AceitouCam1", IntegerType(), True),
        StructField("AceitouCam2", IntegerType(), True),
        StructField("Reclamacao", IntegerType(), True),
        StructField("CustoContato", IntegerType(), True),
        StructField("Receita", IntegerType(), True),
        StructField("UltimaCampanha", IntegerType(), True),
    ])
)

In [275]:
df2= (
    spark
       .read
       .format("csv")
       .option("header", "true")
       .option("delimiter", ",")
       .load('gs://soulcodeprojeto/marketing_campaign.csv',schema=schema)
)

In [276]:
df2.show()
print(df2.count(), len(df2.columns))

+----+-------------+-------------+-----------+-------+----------+--------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+------------+-------+--------------+
|  ID|AnoNascimento|GrauInstrução|EstadoCivil|  Renda|NumCrianca|NumAdolescente| DataConta|UltimaCompra|QtdVinho|QtdFrutas|QtdCarne|QtdPeixe|QtdDoces|QtdOuro|QtdComprasDesconto|QtdComprasWeb|QtdCompraCatalogo|QtdCompraLoja|NumVisitasSite|AceitouCam3|AceitouCam4|AceitouCam5|AceitouCam1|AceitouCam2|Reclamacao|CustoContato|Receita|UltimaCampanha|
+----+-------------+-------------+-----------+-------+----------+--------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------

In [ ]:
a=[x[0] for x in df2.select('GrauInstrução').distinct().collect()]
df2=df2.replace(a, ['Segundo Ciclo', 'PhD', 'Mestrado', 'Graduado', 'Básico'])
df2.show()

In [128]:
b=[x[0] for x in df2.select('EstadoCivil').distinct().collect()]
df2=df2.replace(b,['Só se vive uma vez', 'Juntado', 'Casado', 'Absurdo', 'Viúvo','Divorciado', 'Sozinho', 'Solteiro'])
df2.show()

+----+-------------+-------------+-----------+-------+----------+--------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+------------+-------+--------------+
|  ID|AnoNascimento|GrauInstrução|EstadoCivil|  Renda|NumCrianca|NumAdolescente| DataConta|UltimaCompra|QtdVinho|QtdFrutas|QtdCarne|QtdPeixe|QtdDoces|QtdOuro|QtdComprasDesconto|QtdComprasWeb|QtdCompraCatalogo|QtdCompraLoja|NumVisitasSite|AceitouCam3|AceitouCam4|AceitouCam5|AceitouCam1|AceitouCam2|Reclamacao|CustoContato|Receita|UltimaCampanha|
+----+-------------+-------------+-----------+-------+----------+--------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------

In [129]:
df2.select([count(when(isnan(c), c)).alias(c) for c in df2.columns]).show(vertical=True)

-RECORD 0-----------------
 ID                 | 0   
 AnoNascimento      | 0   
 GrauInstrução      | 0   
 EstadoCivil        | 0   
 Renda              | 0   
 NumCrianca         | 0   
 NumAdolescente     | 0   
 DataConta          | 0   
 UltimaCompra       | 0   
 QtdVinho           | 0   
 QtdFrutas          | 0   
 QtdCarne           | 0   
 QtdPeixe           | 0   
 QtdDoces           | 0   
 QtdOuro            | 0   
 QtdComprasDesconto | 0   
 QtdComprasWeb      | 0   
 QtdCompraCatalogo  | 0   
 QtdCompraLoja      | 0   
 NumVisitasSite     | 0   
 AceitouCam3        | 0   
 AceitouCam4        | 0   
 AceitouCam5        | 0   
 AceitouCam1        | 0   
 AceitouCam2        | 0   
 Reclamacao         | 0   
 CustoContato       | 0   
 Receita            | 0   
 UltimaCampanha     | 0   



In [130]:
df2.select([count(when(col(c).isNull(), c)).alias(c) for c in df2.columns]).show(vertical=True)

-RECORD 0-----------------
 ID                 | 0   
 AnoNascimento      | 0   
 GrauInstrução      | 0   
 EstadoCivil        | 0   
 Renda              | 24  
 NumCrianca         | 0   
 NumAdolescente     | 0   
 DataConta          | 0   
 UltimaCompra       | 0   
 QtdVinho           | 0   
 QtdFrutas          | 0   
 QtdCarne           | 0   
 QtdPeixe           | 0   
 QtdDoces           | 0   
 QtdOuro            | 0   
 QtdComprasDesconto | 0   
 QtdComprasWeb      | 0   
 QtdCompraCatalogo  | 0   
 QtdCompraLoja      | 0   
 NumVisitasSite     | 0   
 AceitouCam3        | 0   
 AceitouCam4        | 0   
 AceitouCam5        | 0   
 AceitouCam1        | 0   
 AceitouCam2        | 0   
 Reclamacao         | 0   
 CustoContato       | 0   
 Receita            | 0   
 UltimaCampanha     | 0   



In [131]:
df2=df2.dropna()

In [132]:
df2.describe().show()

+-------+----------------+------------------+-------------+-----------+------------------+------------------+------------------+----------+-----------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+-----------------+-----------------+-------------------+------------------+-------------------+-------------------+--------------------+-------------------+------------+-------+-------------------+
|summary|              ID|     AnoNascimento|GrauInstrução|EstadoCivil|             Renda|        NumCrianca|    NumAdolescente| DataConta|     UltimaCompra|          QtdVinho|         QtdFrutas|          QtdCarne|         QtdPeixe|          QtdDoces|          QtdOuro|QtdComprasDesconto|     QtdComprasWeb| QtdCompraCatalogo|    QtdCompraLoja|   NumVisitasSite|        AceitouCam3|       AceitouCam4|        AceitouCam5|        AceitouCam1|         AceitouCam2|         Recla

In [133]:
df2=df2.drop('Receita', 'CustoContato')

In [134]:
df2.summary().show()

+-------+----------------+------------------+-------------+-----------+------------------+------------------+------------------+----------+-----------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+-----------------+-----------------+-------------------+------------------+-------------------+-------------------+--------------------+-------------------+-------------------+
|summary|              ID|     AnoNascimento|GrauInstrução|EstadoCivil|             Renda|        NumCrianca|    NumAdolescente| DataConta|     UltimaCompra|          QtdVinho|         QtdFrutas|          QtdCarne|         QtdPeixe|          QtdDoces|          QtdOuro|QtdComprasDesconto|     QtdComprasWeb| QtdCompraCatalogo|    QtdCompraLoja|   NumVisitasSite|        AceitouCam3|       AceitouCam4|        AceitouCam5|        AceitouCam1|         AceitouCam2|         Reclamacao|     UltimaCamp

In [135]:
print(df2.count(), len(df2.columns))

2216 27


In [136]:
df2=df2.withColumnRenamed('NumCrianca','QtdCriancas').withColumnRenamed('NumAdolescente', 'QtdAdolescentes')
df2.show()

+----+-------------+-------------+-----------+-------+-----------+---------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+--------------+
|  ID|AnoNascimento|GrauInstrução|EstadoCivil|  Renda|QtdCriancas|QtdAdolescentes| DataConta|UltimaCompra|QtdVinho|QtdFrutas|QtdCarne|QtdPeixe|QtdDoces|QtdOuro|QtdComprasDesconto|QtdComprasWeb|QtdCompraCatalogo|QtdCompraLoja|NumVisitasSite|AceitouCam3|AceitouCam4|AceitouCam5|AceitouCam1|AceitouCam2|Reclamacao|UltimaCampanha|
+----+-------------+-------------+-----------+-------+-----------+---------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+--------------+
|5524|         1957

In [137]:
df2=df2.withColumn('TotalGasto', F.col('QtdVinho')+F.col('QtdFrutas')+F.col('QtdCarne')+F.col('QtdPeixe')+F.col('QtdOuro')).orderBy('TotalGasto', ascending=False)

In [138]:
df2=df2.withColumn('Filhos', F.col('QtdCriancas')+F.col('QtdAdolescentes')).orderBy('Filhos', ascending=False)

In [139]:
df2.groupBy("EstadoCivil").agg({'QtdVinho':'mean','QtdDoces':'mean' }).show()

+------------------+------------------+------------------+
|       EstadoCivil|     avg(QtdVinho)|     avg(QtdDoces)|
+------------------+------------------+------------------+
|             Viúvo|367.13157894736844| 37.86842105263158|
|        Divorciado| 324.8448275862069| 26.81896551724138|
|           Sozinho|184.66666666666666|               7.0|
|          Solteiro| 291.3312101910828|27.072186836518046|
|           Absurdo|             355.5|              30.5|
|Só se vive uma vez|             322.0|               3.0|
|            Casado|299.85530921820305|26.751458576429403|
|           Juntado|308.40314136125653|26.232111692844676|
+------------------+------------------+------------------+



In [140]:
df2.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- AnoNascimento: integer (nullable = true)
 |-- GrauInstrução: string (nullable = true)
 |-- EstadoCivil: string (nullable = true)
 |-- Renda: double (nullable = true)
 |-- QtdCriancas: integer (nullable = true)
 |-- QtdAdolescentes: integer (nullable = true)
 |-- DataConta: string (nullable = true)
 |-- UltimaCompra: integer (nullable = true)
 |-- QtdVinho: integer (nullable = true)
 |-- QtdFrutas: integer (nullable = true)
 |-- QtdCarne: integer (nullable = true)
 |-- QtdPeixe: integer (nullable = true)
 |-- QtdDoces: integer (nullable = true)
 |-- QtdOuro: integer (nullable = true)
 |-- QtdComprasDesconto: integer (nullable = true)
 |-- QtdComprasWeb: integer (nullable = true)
 |-- QtdCompraCatalogo: integer (nullable = true)
 |-- QtdCompraLoja: integer (nullable = true)
 |-- NumVisitasSite: integer (nullable = true)
 |-- AceitouCam3: integer (nullable = true)
 |-- AceitouCam4: integer (nullable = true)
 |-- AceitouCam5: integer (nullable =

In [141]:
df3 = (df2.withColumn("Status", F.when((F.col('Filhos') >0), F.lit("Filhos em casa")).otherwise(F.lit('Sem Filhos'))))

In [142]:
df3.filter(df3['Status']=='Sem Filhos').show()

+-----+-------------+-------------+-----------+-------+-----------+---------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+--------------+----------+------+----------+
|   ID|AnoNascimento|GrauInstrução|EstadoCivil|  Renda|QtdCriancas|QtdAdolescentes| DataConta|UltimaCompra|QtdVinho|QtdFrutas|QtdCarne|QtdPeixe|QtdDoces|QtdOuro|QtdComprasDesconto|QtdComprasWeb|QtdCompraCatalogo|QtdCompraLoja|NumVisitasSite|AceitouCam3|AceitouCam4|AceitouCam5|AceitouCam1|AceitouCam2|Reclamacao|UltimaCampanha|TotalGasto|Filhos|    Status|
+-----+-------------+-------------+-----------+-------+-----------+---------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+---------

In [143]:
df3.groupBy('EstadoCivil').count().orderBy('count', ascending=False).show()

+------------------+-----+
|       EstadoCivil|count|
+------------------+-----+
|            Casado|  857|
|           Juntado|  573|
|          Solteiro|  471|
|        Divorciado|  232|
|             Viúvo|   76|
|           Sozinho|    3|
|           Absurdo|    2|
|Só se vive uma vez|    2|
+------------------+-----+



In [206]:
df3.groupBy('GrauInstrução').count().show()

+-------------+-----+
|GrauInstrução|count|
+-------------+-----+
|Segundo Ciclo|  200|
|          PhD|  481|
|     Graduado| 1116|
|     Mestrado|  365|
|       Básico|   54|
+-------------+-----+



In [144]:
df3.groupBy('EstadoCivil').agg(F.mean('Renda').alias('RendaMédia')).orderBy('RendaMédia', ascending=False).withColumn("RendaMédia",round(col('RendaMédia'),2)).show()

+------------------+----------+
|       EstadoCivil|RendaMédia|
+------------------+----------+
|           Absurdo|   72365.5|
|             Viúvo|  56481.55|
|           Juntado|  53245.53|
|        Divorciado|  52834.23|
|            Casado|  51724.98|
|          Solteiro|  50995.35|
|Só se vive uma vez|   48432.0|
|           Sozinho|   43789.0|
+------------------+----------+



In [145]:
df3.groupBy('EstadoCivil').agg(F.mean('Renda').alias('RendaMédia'), F.mean('TotalGasto').alias('GastoMedia')).orderBy('GastoMedia', ascending=False).withColumn("RendaMédia",round(col('RendaMédia'),2)).withColumn("GastoMedia",round(col('GastoMedia'),2)).show()

+------------------+----------+----------+
|       EstadoCivil|RendaMédia|GastoMedia|
+------------------+----------+----------+
|           Absurdo|   72365.5|    1162.0|
|             Viúvo|  56481.55|    690.09|
|          Solteiro|  50995.35|    585.52|
|        Divorciado|  52834.23|    583.81|
|           Juntado|  53245.53|    582.61|
|            Casado|  51724.98|    564.71|
|Só se vive uma vez|   48432.0|     421.0|
|           Sozinho|   43789.0|    249.67|
+------------------+----------+----------+



In [146]:
df3=df3.withColumn('Idade', year(current_date())-F.col('AnoNascimento'))

In [207]:
df3.groupBy('Idade').count().orderBy('Idade', ascending=False).show(100)

+-----+-----+
|Idade|count|
+-----+-----+
|  129|    1|
|  123|    1|
|  122|    1|
|   82|    1|
|   81|    1|
|   79|    6|
|   78|    7|
|   77|    8|
|   76|   16|
|   75|   16|
|   74|   21|
|   73|   30|
|   72|   29|
|   71|   42|
|   70|   52|
|   69|   35|
|   68|   49|
|   67|   48|
|   66|   55|
|   65|   41|
|   64|   52|
|   63|   50|
|   62|   49|
|   61|   35|
|   60|   44|
|   59|   44|
|   58|   41|
|   57|   74|
|   56|   50|
|   55|   44|
|   54|   51|
|   53|   70|
|   52|   75|
|   51|   86|
|   50|   78|
|   49|   72|
|   48|   69|
|   47|   83|
|   46|   89|
|   45|   52|
|   44|   76|
|   43|   53|
|   42|   39|
|   41|   38|
|   40|   44|
|   39|   41|
|   38|   38|
|   37|   32|
|   36|   41|
|   35|   27|
|   34|   29|
|   33|   29|
|   32|   18|
|   31|   15|
|   30|   13|
|   29|    5|
|   28|    3|
|   27|    5|
|   26|    2|
+-----+-----+



In [236]:
df4=df3.select('GrauInstrução', 'Idade', 'Renda').withColumn('FaixaIdade', F.when((col('Idade')>70), lit('>70'))
                                                   .when((col('Idade')>60), lit('60-70'))
                                                   .when((col('Idade')>50), lit('50-60'))
                                                   .when((col('Idade')>40), lit('40-50'))
                                                   .when((col('Idade')>30), lit('30-40'))
                                                   .when((col('Idade')>20), lit('>20')))
df4.show()

+-------------+-----+-------+----------+
|GrauInstrução|Idade|  Renda|FaixaIdade|
+-------------+-----+-------+----------+
|     Graduado|   44|60199.0|     40-50|
|     Graduado|   68|93404.0|     60-70|
|     Mestrado|   64|30523.0|     60-70|
|     Graduado|   52|74854.0|     50-60|
|     Graduado|   56|43482.0|     50-60|
|     Graduado|   69|48686.0|     60-70|
|     Graduado|   50|38988.0|     40-50|
|Segundo Ciclo|   49|35688.0|     40-50|
|     Graduado|   66|26150.0|     60-70|
|     Graduado|   63|38285.0|     60-70|
|     Graduado|   55|22574.0|     50-60|
|          PhD|   65|40737.0|     60-70|
|     Mestrado|   57|35791.0|     50-60|
|          PhD|   72|54432.0|       >70|
|     Graduado|   60|46102.0|     50-60|
|          PhD|   67|55521.0|     60-70|
|          PhD|   56|46734.0|     50-60|
|          PhD|   63|33762.0|     60-70|
|     Mestrado|   57|56962.0|     50-60|
|     Graduado|   55|22574.0|     50-60|
+-------------+-----+-------+----------+
only showing top

In [149]:
df5=df4.groupBy('GrauInstrução').pivot('FaixaIdade').mean('Renda').fillna(0)
df5.select(['GrauInstrução','>20','30-40','40-50','50-60','60-70','>70' ]).show()

+-------------+-------+------------------+------------------+------------------+-----------------+------------------+
|GrauInstrução|    >20|             30-40|             40-50|             50-60|            60-70|               >70|
+-------------+-------+------------------+------------------+------------------+-----------------+------------------+
|Segundo Ciclo|64593.5| 39176.28947368421|           41534.7| 53663.17647058824|58657.27027027027| 65988.85714285714|
|          PhD|    0.0|58421.519230769234|         53095.352|53702.156028368794|58119.43010752688| 62199.24285714285|
|     Graduado|65198.0| 45467.30357142857| 52055.64652567976| 52530.02380952381|56757.25306122449|57529.360655737706|
|     Mestrado|67764.5| 45843.53658536585|49641.868131868134| 52108.31428571429|57171.58139534884|         59856.175|
|       Básico|14635.2|           19429.2|           21076.0|           21221.0|          21352.2|           27693.0|
+-------------+-------+------------------+--------------

In [259]:
w0=Window.partitionBy(F.col('EstadoCivil'))
w1=Window.partitionBy(F.col('EstadoCivil')).orderBy('ID')
a=df3.withColumn('rank', F.rank().over(w1)).withColumn('Web',F.sum('QtdComprasWeb').over(w0)).withColumn('Catalogo', F.sum('QtdCompraCatalogo').over(w0)).withColumn('Loja', F.sum('QtdCompraLoja').over(w0)).where(F.col('rank')==1).select('EstadoCivil', 'Web', 'Catalogo', 'Loja')
a.sort('Web').show()

+------------------+----+--------+----+
|       EstadoCivil| Web|Catalogo|Loja|
+------------------+----+--------+----+
|           Absurdo|   7|      15|  13|
|Só se vive uma vez|  14|       2|  12|
|           Sozinho|  15|       2|  12|
|             Viúvo| 351|     251| 483|
|        Divorciado|1000|     620|1350|
|          Solteiro|1814|    1240|2674|
|           Juntado|2351|    1535|3298|
|            Casado|3501|    2254|5013|
+------------------+----+--------+----+



In [151]:
df2.createOrReplaceTempView("clientes")

In [190]:
df5=df3.select('GrauInstrução', 'Idade', 'Renda', 'Reclamacao').withColumn('FaixaIdade', F.when((col('Idade')>70), lit('>70'))
                                                   .when((col('Idade')>60), lit('60-70'))
                                                   .when((col('Idade')>50), lit('50-60'))
                                                   .when((col('Idade')>40), lit('40-50'))
                                                   .when((col('Idade')>30), lit('30-40'))
                                                   .when((col('Idade')>20), lit('>20')))

In [228]:
win0=Window.partitionBy(F.col('FaixaIdade'))
win1=Window.partitionBy(F.col('FaixaIdade')).orderBy('Reclamacao')
df6=df5.withColumn('rank', F.row_number().over(win1)).withColumn('TotalReclamação',sum('Reclamacao').over(win0)).where(F.col('rank') == 1).select('FaixaIdade', 'TotalReclamação')
df6.orderBy('TotalReclamação', ascending=False).show()

+----------+---------------+
|FaixaIdade|TotalReclamação|
+----------+---------------+
|     60-70|              6|
|     40-50|              6|
|       >70|              4|
|     30-40|              3|
|     50-60|              1|
|       >20|              1|
+----------+---------------+



In [226]:
df5.groupBy('FaixaIdade').agg(sum('Reclamacao').alias('TotalReclamação')).orderBy('TotalReclamação', ascending=False).show()

+----------+---------------+
|FaixaIdade|TotalReclamação|
+----------+---------------+
|     60-70|              6|
|     40-50|              6|
|       >70|              4|
|     30-40|              3|
|       >20|              1|
|     50-60|              1|
+----------+---------------+



In [265]:
###SQL
df3.createOrReplaceTempView('clientes')
spark.sql("SELECT * FROM clientes").show()

+-----+-------------+-------------+-----------+-------+-----------+---------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+--------------+----------+------+--------------+-----+
|   ID|AnoNascimento|GrauInstrução|EstadoCivil|  Renda|QtdCriancas|QtdAdolescentes| DataConta|UltimaCompra|QtdVinho|QtdFrutas|QtdCarne|QtdPeixe|QtdDoces|QtdOuro|QtdComprasDesconto|QtdComprasWeb|QtdCompraCatalogo|QtdCompraLoja|NumVisitasSite|AceitouCam3|AceitouCam4|AceitouCam5|AceitouCam1|AceitouCam2|Reclamacao|UltimaCampanha|TotalGasto|Filhos|        Status|Idade|
+-----+-------------+-------------+-----------+-------+-----------+---------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-

In [269]:
spark.sql("SELECT count(Renda) from clientes\
          where Renda>50000").show()

+------------+
|count(Renda)|
+------------+
|        1156|
+------------+



In [279]:
spark.sql("SELECT * from clientes\
        where UltimaCompra>60").show()

+-----+-------------+-------------+-----------+-------+-----------+---------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-----------+-----------+-----------+----------+--------------+----------+------+--------------+-----+
|   ID|AnoNascimento|GrauInstrução|EstadoCivil|  Renda|QtdCriancas|QtdAdolescentes| DataConta|UltimaCompra|QtdVinho|QtdFrutas|QtdCarne|QtdPeixe|QtdDoces|QtdOuro|QtdComprasDesconto|QtdComprasWeb|QtdCompraCatalogo|QtdCompraLoja|NumVisitasSite|AceitouCam3|AceitouCam4|AceitouCam5|AceitouCam1|AceitouCam2|Reclamacao|UltimaCampanha|TotalGasto|Filhos|        Status|Idade|
+-----+-------------+-------------+-----------+-------+-----------+---------------+----------+------------+--------+---------+--------+--------+--------+-------+------------------+-------------+-----------------+-------------+--------------+-----------+-----------+-